In [30]:
import numpy as np
import pandas as pd
import statsmodels as stats
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
df_ww2 = pd.read_csv('data/operations.csv')


C:\Users\maria\AppData\Local\Temp\ipykernel_26084\2468670380.py:1: DtypeWarning: Columns (7,8,11,13,31,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ww2 = pd.read_csv('data/operations.csv')


In [46]:
df_ww2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178281 entries, 0 to 178280
Data columns (total 46 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Mission ID                             178281 non-null  int64  
 1   Mission Date                           178281 non-null  object 
 2   Theater of Operations                  175123 non-null  object 
 3   Country                                126494 non-null  object 
 4   Air Force                              126444 non-null  object 
 5   Unit ID                                50913 non-null   object 
 6   Aircraft Series                        178165 non-null  object 
 7   Callsign                               21 non-null      object 
 8   Mission Type                           47186 non-null   object 
 9   Takeoff Base                           3383 non-null    object 
 10  Takeoff Location                       3338 non-null    

In [35]:
description = df_ww2.describe().transpose()
description['CV'] = description.apply(lambda x: 100*(x['std']/x['mean']), axis=1)
##description['NaN_count'] = description.apply(lambda x: df_ww2[x].isna().sum(), axis=1)

lista_columnas = description.index.to_list()
nan_column = [df_ww2[column].isna().sum() for column in lista_columnas]
description['nan_count'] = nan_column
description['nan %'] = description.apply(lambda x: np.round(100*(x['nan_count']/(x['count'] + x['nan_count'])), 2), axis=1)
description['IQR'] = description.apply(lambda x: np.round(x['75%']-x['25%'], 2), axis=1)
kurtosis_column = [np.round(kurtosis(a= df_ww2[column], nan_policy='omit'), 4) for column in lista_columnas]
description['kurtosis'] = kurtosis_column
skew_column = [np.round(skew(a= df_ww2[column], nan_policy='omit'), 4) for column in lista_columnas]
description['skew'] = skew_column
description

,count,mean,std,min,25%,50%,75%,max,CV,nan_count,nan %,IQR,kurtosis,skew
Mission ID,178281.0,89414.547041,51649.744549,1.000000,44571.00,89540.000000,134171.000000,178742.00,57.764364,0,0.00,89600.00,-1.2028,-0.0021
Takeoff Longitude,2980.0,220.618627,454.046956,-177.328611,14.40,29.851389,96.133222,1355.00,205.806265,175301,98.33,81.73,2.3682,2.0715
Target Latitude,169565.0,38.067793,18.711405,-37.416667,26.25,47.500000,50.930000,1108.00,49.152850,8716,4.89,24.68,63.2699,-0.1009
Target Longitude,169543.0,43.340213,59.681160,-179.451285,7.62,11.330000,97.400000,1000.65,137.703893,8738,4.90,89.78,43.0639,3.5303
Altitude (Hundreds of Feet),120826.0,139.026812,186.818278,0.000000,70.00,122.000000,223.000000,40000.00,134.375719,57455,32.23,153.00,19980.4558,112.4662
Airborne Aircraft,82242.0,12.329771,12.514833,0.000000,2.00,11.000000,17.000000,332.00,101.500941,96039,53.87,15.00,19.0710,2.4961
Attacking Aircraft,125108.0,10.218899,11.246399,0.000000,2.00,7.000000,12.000000,332.00,110.054899,53173,29.83,10.00,25.6268,3.1986
Bombing Aircraft,81612.0,12.347486,12.573549,0.000000,2.00,11.000000,18.000000,332.00,101.830846,96669,54.22,16.00,19.0171,2.4974
Aircraft Returned,133.0,2.157895,1.369816,1.000000,1.00,2.000000,3.000000,7.00,63.479272,178148,99.93,2.00,1.3658,1.2933
Aircraft Failed,247.0,4.275304,5.924758,1.000000,1.00,2.000000,4.000000,48.00,138.580996,178034,99.86,3.00,19.7124,3.8210


In [15]:
print(df_ww2['Aircraft Damaged'].isna().sum() / len(df_ww2['Aircraft Damaged']))
print(df_ww2['Aircraft Damaged'].count())
print(df_ww2['Aircraft Damaged'].info())

0.9997868533382693
38
<class 'pandas.core.series.Series'>
RangeIndex: 178281 entries, 0 to 178280
Series name: Aircraft Damaged
Non-Null Count  Dtype  
--------------  -----  
38 non-null     float64
dtypes: float64(1)
memory usage: 1.4 MB
None


In [5]:
data_iris = sns.load_dataset('iris')
data_iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [6]:
from pyobsplot import Plot


In [7]:

Plot.plot(
    {
        "grid": True,
        "marks": [
            Plot.dot(
                data_iris,
                {
                    "x": "sepal_width",
                    "y": "petal_length",
                    "fill": "species",
                    "tip": True,
                },
            )
        ],
    }
)

ObsplotWidget(spec={'data': [{'pyobsplot-type': 'DataFrame', 'value': b'ARROW1\x00\x00\xff\xff\xff\xff(\x05\x0…

In [36]:
def describe_plus(dataset):
    from scipy.stats import kurtosis, skew
    description = dataset.describe().transpose()
    description['CV'] = description.apply(lambda x: 100*(x['std']/x['mean']), axis=1)
    lista_columnas = description.index.to_list()
    nan_column = [df_ww2[column].isna().sum() for column in lista_columnas]
    description['nan_count'] = nan_column
    description['nan %'] = description.apply(lambda x: np.round(100*(x['nan_count']/(x['count'] + x['nan_count'])), 2), axis=1)
    description['IQR'] = description.apply(lambda x: np.round(x['75%']-x['25%'], 2), axis=1)
    kurtosis_column = [np.round(kurtosis(a= df_ww2[column], nan_policy='omit'), 4) for column in lista_columnas]
    description['kurtosis'] = kurtosis_column
    skew_column = [np.round(skew(a= df_ww2[column], nan_policy='omit'), 4) for column in lista_columnas]
    description['skew'] = skew_column

    return description

In [37]:
describe_plus(df_ww2)

,count,mean,std,min,25%,50%,75%,max,CV,nan_count,nan %,IQR,kurtosis,skew
Mission ID,178281.0,89414.547041,51649.744549,1.000000,44571.00,89540.000000,134171.000000,178742.00,57.764364,0,0.00,89600.00,-1.2028,-0.0021
Takeoff Longitude,2980.0,220.618627,454.046956,-177.328611,14.40,29.851389,96.133222,1355.00,205.806265,175301,98.33,81.73,2.3682,2.0715
Target Latitude,169565.0,38.067793,18.711405,-37.416667,26.25,47.500000,50.930000,1108.00,49.152850,8716,4.89,24.68,63.2699,-0.1009
Target Longitude,169543.0,43.340213,59.681160,-179.451285,7.62,11.330000,97.400000,1000.65,137.703893,8738,4.90,89.78,43.0639,3.5303
Altitude (Hundreds of Feet),120826.0,139.026812,186.818278,0.000000,70.00,122.000000,223.000000,40000.00,134.375719,57455,32.23,153.00,19980.4558,112.4662
Airborne Aircraft,82242.0,12.329771,12.514833,0.000000,2.00,11.000000,17.000000,332.00,101.500941,96039,53.87,15.00,19.0710,2.4961
Attacking Aircraft,125108.0,10.218899,11.246399,0.000000,2.00,7.000000,12.000000,332.00,110.054899,53173,29.83,10.00,25.6268,3.1986
Bombing Aircraft,81612.0,12.347486,12.573549,0.000000,2.00,11.000000,18.000000,332.00,101.830846,96669,54.22,16.00,19.0171,2.4974
Aircraft Returned,133.0,2.157895,1.369816,1.000000,1.00,2.000000,3.000000,7.00,63.479272,178148,99.93,2.00,1.3658,1.2933
Aircraft Failed,247.0,4.275304,5.924758,1.000000,1.00,2.000000,4.000000,48.00,138.580996,178034,99.86,3.00,19.7124,3.8210


In [48]:
df_ww2['Fragmentation Devices Type'].value_counts()

Fragmentation Devices Type
120 LB FRAG (6X20 CLUSTERS)                1984
260 LB FRAG                                1209
69 LB FRAG (3X23 CLUSTERS)                  805
20 LB FRAG                                  617
23 LB FRAG                                  513
400 LB FRAG (20X20 CLUSTERS)                371
540 LB FRAG (6X90 CLUSTERS)                  53
4 LB FRAG                                    49
17 KG CHINESE                                39
96 LB FRAG (24X4 CLUSTERS)                   25
90 LB FRAG                                   24
23 LB PARAFRAG                               20
360 LB FRAG (90X4 CLUSTERS)                  17
30 LB FRAG                                   15
20 LB PARA FRAG                               6
40 LB FRAG                                    5
250 LB FRAG                                   3
35 LB FRAG                                    3
UNK CODE 15                                   2
23 LB FRAG CLUSTERS (6 X23 PER CLUSTER)       2
15 KG CHINESE